In [ ]:
import numpy as np

a = np.array([[[1,7,13],[3,9,15],[5,11,17]],[[2,8,14],[4,10,16],[6,12,18]]])
a_tr = a[0,:].transpose()
#print(a_tr.reshape(3,-1))
v3_input = a_tr.reshape(3,-1)

print(v3_input)
#while(1):


[[ 1  3  5]
 [ 7  9 11]
 [13 15 17]]


In [ ]:
def fourDTCto2D(g3_arr):
  #print(g3_arr)
  arr_shape = g3_arr.shape
  init_rflag=1
  for r_data in g3_arr:
    init_cflag=1
    for c_data in r_data:
      if(init_cflag):
        mat2d=c_data
        init_cflag=0
      else:
        mat2d = np.concatenate((mat2d,c_data),axis=1)
    if(init_rflag):
        total_mat=mat2d
        init_rflag=0
    else:
      total_mat = np.concatenate((total_mat,mat2d),axis=0)

  return total_mat

g3_arr = np.array([[[[1],[0],[-1]],[[2],[0],[-2]],[[3],[0],[-3]]],[[[4],[0],[-4]],[[5],[0],[-5]],[[6],[0],[-6]]]])
print(g3_arr.shape)
g2_arr = np.array([[[[1,2,3],[0,1,-4],[-1,-2,0]],[[4,5,6],[7,-1,0],[-2,0,0]],[[7,8,9],[-2,-1,-3],[0,2,0]]]])
print(g2_arr.shape)

g3_2d=fourDTCto2D(g3_arr)
print(g3_2d)
g2_2d=fourDTCto2D(g2_arr)
print(g2_2d)


(2, 3, 3, 1)
(1, 3, 3, 3)
[[ 1  2  3]
 [ 0  0  0]
 [-1 -2 -3]
 [ 4  5  6]
 [ 0  0  0]
 [-4 -5 -6]]
[[ 1  2  3  4  5  6  7  8  9]
 [ 0  1 -4  7 -1  0 -2 -1 -3]
 [-1 -2  0 -2  0  0  0  2  0]]


In [ ]:
def transform(arr,m_list,n_list,h,r):
  arr_tr = np.transpose(arr)
  #print(arr_tr)
  arr_re = arr_tr.reshape(n_list[h-1],-1)
  new_arr_width = len(arr_re[0])
  #print(new_arr_width)
  new_arr=[]
  #print(arr_re)
  #print(arr_re[:,0:4])
  #r = 3
  p = new_arr_width
  for i in range(0,p,r):
    temp = arr_re[:,i:i+r]
    new_arr.append(temp.reshape(-1))

  return np.transpose(new_arr)

In [ ]:
g1_arr=np.array([[[[2,1,-1]],[[3,4,-3]]],[[[-2,-1,-1]],[[0,0,-1]]]])
print(g1_arr.shape)
g1_2d=fourDTCto2D(g1_arr)
print(g1_2d)

(2, 2, 1, 3)
[[ 2  1 -1  3  4 -3]
 [-2 -1 -1  0  0 -1]]


In [ ]:
print(np.matmul(g1_2d[:,0:0+1*3],[[-396]
 ,[ 756]
 ,[-186]]))

print(np.matmul(g1_2d[:,3:3+1*3],[[-432]
 ,[ 828]
 ,[-204]]))

print(np.matmul(g1_2d[:,0:0+1*3],[[-882],
 [1674],
 [-411]]))

print(np.matmul(g1_2d[:,3:3+1*3],[[-972]
 ,[1854]
 ,[-456]]))

[[150]
 [222]]
[[2628]
 [ 204]]
[[321]
 [501]]
[[5868]
 [ 456]]


In [ ]:
'''
b50=[]
a = np.matmul(g1_2d[:,0:0+1*3],[[-396]
 ,[ 756]
 ,[-186]])
b = np.matmul(g1_2d[:,3:3+1*3],[[-432]
 ,[ 828]
 ,[-204]])
c = np.matmul(g1_2d[:,0:0+1*3],[[-882],
 [1674],
 [-411]])
d = np.matmul(g1_2d[:,3:3+1*3],[[-972]
 ,[1854]
 ,[-456]])
'''

In [ ]:
m_list=[2,1,2]
n_list=[2,3,3]
r = 3

s = 3
#r0 = np.matmul(g3_2d[s:s+r],v3_input)
#print(r0)
#v2_input = transform(r0,m_list,n_list,2,r)
#print(v2_input)

cycle=0
slice_point=0
slice_range=1
#- Set some hardware configuration
cycle=0
final_flag=0
TC_finish=[0,0,0]
input_i=0
g3_start=0
f_tc_cnt=0

v3_input=[]
buf_3to2=[[],[]]
buf_2to1=[[],[]]
accu_buf=[]

r1=[]
r2=[]
r3=[]

while(1):
  cycle=cycle+1
  print('--------------------------')
  if(final_flag or cycle >= 10):
    print('TOTAL CYCLE: ',cycle-1)
    break

  #----stage3(LAST)--Tensor_Core1--process-----START
  if(slice_point==g1_2d.shape[1]):
    slice_point=0
    final_acc = np.array(accu_buf)
    # Final Accumulation in Buffer RRAM
    print('******',np.sum(final_acc,axis=0))
    accu_buf=[]
    if(f_tc_cnt == n_list[0]-1):
      TC_finish[2]=1
    else:
      f_tc_cnt=f_tc_cnt+1

  if len(buf_2to1[0]):
    r1 = np.matmul(g1_2d[:,slice_point:slice_point+slice_range*r],buf_2to1[0])
    print(r1,'from buf_2to1[0]')
    print(cycle,'from buf_2to1[0]')
    buf_2to1[0]=[]
    slice_point = slice_point+slice_range*r
    accu_buf.append(r1)

  if len(buf_2to1[1]):
    r1 = np.matmul(g1_2d[:,slice_point:slice_point+slice_range*r],buf_2to1[0])
    print(r1,'from buf_2to1[0]')
    print(cycle,'from buf_2to1[0]')
    buf_2to1[1]=[]
    slice_point = slice_point+slice_range*r
    accu_buf.append(r1)

  #----stage3(LAST)--Tensor_Core1--process-----END
  # accumulate buffer


  #----stage2--Tensor_Core2--process-----START
  if len(buf_3to2[0]):
    r2 = np.matmul(g2_2d,buf_3to2[0])
    print(r2,'from buf_3to2[0]')
    print(cycle)
    buf_3to2[0]=[]

  if len(buf_3to2[1]):
    r2 = np.matmul(g3_2d,buf_3to2[1])
    print(r2,'from buf_3to2[1]')
    print(cycle)
    buf_3to2[1]=[]


  if len(buf_2to1[0])==0 and len(r2):
    buf_2to1[0] = transform(r2,m_list,[1,3,3],1,r)
    r2=[]

  elif len(buf_2to1[1])==0 and len(r2):
    buf_2to1[1] = transform(r2,m_list,[1,3,3],1,r)
    r2=[]
  else:
    print('TensorCore2 END')
    TC_finish[1]=1
    #break
  #----stage2--Tensor_Core2--process-----END



  #----stage1--Tensor_Core3--process-----START
  #----get initial input-------
  if(g3_start < g3_2d.shape[0]):
    a = np.array([[[1,7,13],[3,9,15],[5,11,17]],[[2,8,14],[4,10,16],[6,12,18]]])
    a_tr = a[input_i,:].transpose()
    v3_input = a_tr.reshape(3,-1)
    #print(v3_input)



  #print(g3_2d[g3_start:g3_start+r])
  if len(v3_input):
    r3 = np.matmul(g3_2d[g3_start:g3_start+r],v3_input)
    #print('1111111',r3)
    v3_input=[]

  #print(r3)
  #v2_input = transform(r3,m_list,n_list,2,r)
  #if not(buf_3to2[0]):
  if len(buf_3to2[0])==0 and len(r3):
    buf_3to2[0] = transform(r3,m_list,n_list,2,r)
    #print('0000000',r3)
    r3=[]

  elif len(buf_3to2[1])==0 and len(r3):
    buf_3to2[1] = transform(r3,m_list,n_list,2,r)
    #print('00000000',r3)
    r3=[]
  else:
    print('TensorCore3 END')
    TC_finish[0]=1
    #break

  #----stage1--Tensor_Core3--process-----END

  #----choose input slice--------start
  if (input_i < n_list[0]-1):
    input_i = input_i + 1
  else:
    input_i = 0
    g3_start = g3_start + r

  #----choose input slice--------end

  if(TC_finish[0] and TC_finish[1] and TC_finish[2]):
    print('ALL finish')
    final_flag=1




--------------------------
TensorCore2 END
--------------------------
[[-396]
 [ 756]
 [-186]] from buf_3to2[0]
2
--------------------------
[[150]
 [222]] from buf_2to1[0]
3 from buf_2to1[0]
[[-432]
 [ 828]
 [-204]] from buf_3to2[0]
3
--------------------------
[[2628]
 [ 204]] from buf_2to1[0]
4 from buf_2to1[0]
[[-882]
 [1674]
 [-411]] from buf_3to2[0]
4
--------------------------
****** [[2778]
 [ 426]]
[[321]
 [501]] from buf_2to1[0]
5 from buf_2to1[0]
[[-972]
 [1854]
 [-456]] from buf_3to2[0]
5
TensorCore3 END
--------------------------
[[5868]
 [ 456]] from buf_2to1[0]
6 from buf_2to1[0]
TensorCore2 END
TensorCore3 END
--------------------------
****** [[6189]
 [ 957]]
TensorCore2 END
TensorCore3 END
ALL finish
--------------------------
TOTAL CYCLE:  7
